In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from tqdm.notebook import tqdm
from result_saver import SaverProvider
from Scratch import metadata_loader
provider = SaverProvider()

In [3]:
df = metadata_loader(_extract=True)
#df = df[df.job_status != 'JobStatus.QUEUED']


In [4]:
from time import sleep

start_idx = 0 # Initialize starting index

while True:  # Outer loop to restart when a specific error occurs
    try:
        for idx, job_id in enumerate(tqdm(df.job_id[start_idx:])):

            provider.retrieve_job(job_id, _ignore_running=False)

            start_idx = start_idx + 1  
        break  
    except KeyboardInterrupt:
        print('Interrupted')
        break  
    except Exception as e:
        if 'RequestsApiError' in str(type(e)):
            print(f'\nRequestsApiError occurred at job index {start_idx}, retrying...')
            sleep(5) 
        else:
            print(f'An unexpected error occurred: {e}')
            break  


  0%|          | 0/2279 [00:00<?, ?it/s]

20:17:23 Warning: Job ID cn6j2mtss5h00087k2cg not found in /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
20:17:47 Warning: Job ID cn6j2gaanbvg008dadag not found in /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
20:17:56 Warning: Job ID cn6j2gaanbvg008dadag is still running. Aborting.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 86
20:17:56 Warning: Job ID cn6j2chss5

  0%|          | 0/2190 [00:00<?, ?it/s]

20:42:36 Warning: Job ID cn6hp48anbvg008dacbg not found in /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
20:42:49 Warning: Job ID cn6hp48anbvg008dacbg is still running. Aborting.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 86
20:42:49 Warning: Job ID cn6hny7anbvg008daca0 not found in /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
20:43:01 Warning: Job ID cn6hny7anb

  0%|          | 0/2159 [00:00<?, ?it/s]

20:50:52 Warning: Job ID cn6hgvk9recg008x0jn0 not found in /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
20:51:16 Warning: Job ID cn6hgp262r90008815a0 not found in /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
20:51:26 Warning: Job ID cn6hgp262r90008815a0 is still running. Aborting.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 86
20:51:26 Warning: Job ID cn6hggjanb

  0%|          | 0/2154 [00:00<?, ?it/s]

20:52:19 Warning: Job ID cn6hg1gxhnxg008djptg not found in /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
20:52:25 Warning: Job ID cn6hg1gxhnxg008djptg is in an unknown state. Updating Metadata & Aborting.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 92
20:52:25 Warning: Job ID cn6hfwfxhnxg008djpsg not found in /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
20:52:31 

In [ ]:
df = df[df['job_id']=="cmxnz9dwjay0008cnwc0"]
if df.job_status.values == 'JobStatus.DONE':
    print("Job is done")    